# ETL Pipeline for music and fashion data

### Extract (E)
- Download Kaggle Datasets
- Store raw datasets in /datasets/raw/x_data

### Transform (T)
- Clean data, if necessary
- Standardize formats; dates, capitalization, classes
- Remove non-overlapping dates, only keep rows from both data sets where dates overlap

### Load (L)
- Store processed datasets in /datasets/process/x_data

In [ ]:
import pandas as pd
from scripts.music_info import get_artist_genres, get_track_genres

In [2]:
# Fashion data
customers_df = pd.read_csv('../datasets/raw/fashion_data/customers.csv')
discounts_df = pd.read_csv('../datasets/raw/fashion_data/discounts.csv')
products_df = pd.read_csv('../datasets/raw/fashion_data/products.csv')
transactions_df = pd.read_csv('../datasets/raw/fashion_data/transactions.csv')

# Music data
music_df = pd.read_csv('../datasets/raw/music_data/universal_top_spotify_songs.csv')

C:\Users\penaf\AppData\Local\Temp\ipykernel_52488\4285717369.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  customers_df = pd.read_csv('../datasets/raw/fashion_data/customers.csv')


## Fashion columns to keep

### Customers
- id
- city
- country
- gender
- age (use dob)
- job

In [3]:
customers_df.drop(columns=['Name', 'Email', 'Telephone'], axis=1, inplace=True)
customers_df['Age'] = pd.to_datetime('today').year - pd.to_datetime(customers_df['Date Of Birth'], errors='coerce').dt.year
customers_df.drop(columns=['Date Of Birth'], axis=1, inplace=True)
customers_df['Job Title'].fillna('Unknown', inplace=True)
customers_df.head()

C:\Users\penaf\AppData\Local\Temp\ipykernel_52488\481525094.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers_df['Job Title'].fillna('Unknown', inplace=True)


,Customer ID,City,Country,Gender,Job Title,Age
0,1,New York,United States,M,Unknown,22
1,2,New York,United States,M,Records manager,25
2,3,New York,United States,F,Unknown,22
3,4,New York,United States,M,Proofreader,29
4,5,New York,United States,F,Exercise physiologist,27


In [4]:
customers_df['Country'].value_counts()

Country
United States     354450
中国                340082
España            237575
Deutschland       205560
France            196696
United Kingdom    190574
Portugal          118369
Name: count, dtype: int64

### Discounts

- Discount
- Category (has null)
- Sub Category (has null)
- Season (use start and end)

In [5]:
season_mapping = {
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Fall',
    10: 'Fall',
    11: 'Fall',
    12: 'Winter'
}

discounts_df.fillna({'Category': 'Unknown'}, inplace=True)
discounts_df.fillna({'Sub Category': 'Unknown'}, inplace=True)
discounts_df['Season'] = discounts_df['Start'].apply(lambda x: season_mapping[pd.to_datetime(x, errors='coerce').month] if pd.notnull(x) else 'Unknown')
discounts_df.drop(columns=['Start', 'End', 'Description'], axis=1, inplace=True)

In [6]:
discounts_df.head()

,Discont,Category,Sub Category,Season
0,0.4,Feminine,Coats and Blazers,Winter
1,0.4,Feminine,Sweaters and Knitwear,Winter
2,0.4,Masculine,Coats and Blazers,Winter
3,0.4,Masculine,Sweaters and Sweatshirts,Winter
4,0.4,Children,Coats,Winter


### Products

- Id
- Category
- Sub Category
- Description EN
- Color

In [7]:
products_df.drop(columns=['Description PT', 'Description DE', 'Description FR', 'Description ES', 'Description ZH', 'Sizes', 'Production Cost'], axis=1, inplace=True)
products_df.fillna({'Color': 'Unknown'}, inplace=True)

In [8]:
products_df.head()

,Product ID,Category,Sub Category,Description EN,Color
0,1,Feminine,Coats and Blazers,Sports Velvet Sports With Buttons,Unknown
1,2,Feminine,Sweaters and Knitwear,Luxurious Pink Denim With Buttons,PINK
2,3,Feminine,Dresses and Jumpsuits,Black Tricot Printed Tricot,BLACK
3,4,Feminine,Shirts and Blouses,Basic Cotton Blouse,Unknown
4,5,Feminine,T-shirts and Tops,Basic Cotton T-Shirt,Unknown


### Transactions

- Customer ID
- Product ID
- Size
- Color
- Unit Price
- Quantity
- Date
- Discount
- Store ID
- Currency
- Payment Method
- Invoice Total

In [9]:
transactions_df.drop(columns=['Invoice ID', 'Line', 'Line Total', 'Employee ID', 'Currency Symbol', 'SKU', 'Transaction Type'], axis=1, inplace=True)

In [10]:
transactions_df.head()

,Customer ID,Product ID,Size,Color,Unit Price,Quantity,Date,Discount,Store ID,Currency,Payment Method,Invoice Total
0,47162,485,M,NaN,80.5,1,2023-01-01 15:42:00,0.0,1,USD,Cash,126.7
1,47162,2779,G,NaN,31.5,1,2023-01-01 15:42:00,0.4,1,USD,Cash,126.7
2,47162,64,M,NEUTRAL,45.5,1,2023-01-01 15:42:00,0.4,1,USD,Cash,126.7
3,10142,131,M,BLUE,70.0,1,2023-01-01 20:04:00,0.4,1,USD,Cash,77.0
4,10142,716,L,WHITE,26.0,1,2023-01-01 20:04:00,0.0,1,USD,Cash,77.0


## Music Columns to keep

### Music

- ID
- name
- artists
- daily_rank
- daily_movement
- weekly_movement
- country
- snapshot_date
- popularity
- (all music data)

In [13]:
music_df.head(-1)

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2RkZ5LkEzeHGRsmDqKwmaJ,Ordinary,Alex Warren,1,1,0,NaN,2025-06-11,95,False,...,2,-6.141,1,0.0600,0.704000,0.000007,0.0550,0.391,168.115,3
1,42UBPzRMh5yyz0EDPr6fr1,Manchild,Sabrina Carpenter,2,-1,48,NaN,2025-06-11,89,True,...,7,-5.087,1,0.0572,0.122000,0.000000,0.3170,0.811,123.010,4
2,0FTmksd2dxiE5e3rWyJXs6,back to friends,sombr,3,0,1,NaN,2025-06-11,98,False,...,1,-2.291,1,0.0301,0.000094,0.000088,0.0929,0.235,92.855,4
3,7so0lgd0zP2Sbgs2d7a1SZ,Die With A Smile,"Lady Gaga, Bruno Mars",4,0,-1,NaN,2025-06-11,91,False,...,6,-7.727,0,0.0317,0.289000,0.000000,0.1260,0.498,157.964,3
4,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,5,1,0,NaN,2025-06-11,100,False,...,2,-10.171,1,0.0358,0.200000,0.060800,0.1170,0.438,104.978,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110310,6I3mqTwhRpn34SLVafSH7G,Ghost,Justin Bieber,45,5,0,AE,2023-10-18,88,False,...,2,-5.569,1,0.0478,0.185000,0.000029,0.4150,0.441,153.960,4
2110311,0AYt6NMyyLd0rLuvr0UkMH,Slime You Out (feat. SZA),"Drake, SZA",46,4,0,AE,2023-10-18,84,True,...,5,-9.243,0,0.0502,0.508000,0.000000,0.2590,0.105,88.880,3
2110312,2Gk6fi0dqt91NKvlzGsmm7,SAY MY GRACE (feat. Travis Scott),"Offset, Travis Scott",47,3,0,AE,2023-10-18,80,True,...,10,-5.060,1,0.0452,0.058500,0.000000,0.1320,0.476,121.879,4
2110313,26b3oVLrRUaaybJulow9kz,People,Libianca,48,2,0,AE,2023-10-18,88,False,...,10,-7.621,0,0.0678,0.551000,0.000013,0.1020,0.693,124.357,5
